In [1]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
from scipy.sparse import hstack
from sklearn.pipeline import FeatureUnion

In [2]:
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn import utils
import sklearn

In [3]:
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics
from sklearn import utils
import sklearn
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import xgboost as xgb



In [4]:
from collections import Counter, OrderedDict
import collections

In [5]:
import joblib


In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [7]:
import re

In [8]:
from textblob import TextBlob

Based on https://www.kaggle.com/venomsnake/analyzing-sarcasm-comprehensive-eda-detection and https://github.com/surajr/SarcasmDetection

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
df_isarcasm=pd.read_csv("/content/drive/MyDrive/ISarcasm/DataSet/train.En.csv")

In [11]:
df_isarcasm=df_isarcasm[['tweet','sarcastic']]

In [12]:
train, validate, test = \
              np.split(df_isarcasm.sample(frac=1, random_state=42), 
                       [int(.6*len(df_isarcasm)), int(.8*len(df_isarcasm))])

In [13]:
train_new=pd.concat([train, validate], ignore_index=True)

In [14]:
#data augmentation
!pip install googletrans==4.0.0-rc1
from googletrans import Translator

In [15]:
#Data Augmentation (We have imbalanced data: Sarcastic -> 867, Non-sarcastic -> 2600)

def data_augmentation(sequence, aug_range=1, PROB = 1):
    languages = ['en', 'fr', 'th', 'tr', 'ur', 'ru', 'bg', 'de', 'ar', 'zh-cn', 'hi',
                 'sw', 'vi', 'es', 'el']

    augmented_tweets = []

    #instantiate translator
    translator = Translator()
    
    #store original language so we can convert back
    org_lang = translator.detect(sequence).lang
    
    for i in range(0,aug_range):
      #randomly choose language to translate sequence to  
      random_lang = np.random.choice([lang for lang in languages if lang is not org_lang])
      
      if org_lang in languages:
          #translate to new language and back to original
          translated = translator.translate(sequence, dest = random_lang).text
          #translate back to original language
          translated_back = translator.translate(translated, dest = org_lang).text
      
          #apply with certain probability
          if np.random.uniform(0, 1) <= PROB:
              output_sequence = translated_back
          else:
              output_sequence = sequence
          
          augmented_tweets.append(output_sequence)
          
      #if detected language not in our list of languages, do nothing
      else:
          augmented_tweets.append(sequence)
    
    return augmented_tweets

In [16]:
import os,re,string,nltk,operator,math,tqdm



In [17]:
# Get max intent count to match other minority classes through data augmentation
intent_count = train_new.sarcastic.value_counts().to_dict()
max_intent_count = max(intent_count.items(), key=operator.itemgetter(1))[1]
max_intent_count

2069

In [18]:
## Loop to interate all messages

newdf = pd.DataFrame()
for intent, count in intent_count.items() :
    count_diff = max_intent_count - count    ## Difference to fill
    multiplication_count = math.ceil((count_diff)/count)  ## Multiplying a minority classes for multiplication_count times
    if (multiplication_count) :
        old_message_df = pd.DataFrame()
        new_message_df = pd.DataFrame()
        for message in tqdm.tqdm(train_new[train_new["sarcastic"] == intent]["tweet"]) :
            ## Extracting existing minority class batch
            dummy1 = pd.DataFrame([message], columns=['tweet'])
            dummy1["sarcastic"] = intent
            old_message_df = old_message_df.append(dummy1)
            
            ## Creating new augmented batch from existing minority class
            new_messages = data_augmentation(message, multiplication_count)
            dummy2 = pd.DataFrame(new_messages, columns=['tweet'])
            dummy2["sarcastic"] = intent
            new_message_df = new_message_df.append(dummy2)
        
        ## Select random data points from augmented data
        new_message_df=new_message_df.take(np.random.permutation(len(new_message_df))[:count_diff])
        
        ## Merge existing and augmented data points
        newdf = newdf.append([old_message_df,new_message_df])
    else :
        newdf = newdf.append(train_new[train_new["sarcastic"] == intent])

100%|██████████| 705/705 [37:14<00:00,  3.17s/it]


In [19]:
def remove_punctuations(text:str)->str: 
  return  "".join(char for char in text if char not in string.punctuation)
  

def clean_tweet(text:str)->str:
  '''Should remove hashtags and mentions first before removing punctuations'''
  # remove mentions
  text = re.sub('@[A-Za-z0-9_]+', '', text)

  # remove hashtags
  text = re.sub('#[A-Za-z0-9_]+','', text)

  # remove links
  text = re.sub(r"www.\S+", "", text)
  text = re.sub(r"http\S+", "", text)
  
  # remove the punctuations in text from string.punctuation
  text  = remove_punctuations(text)

  # filter only alphanumeric characters except whitespaces
  text = re.sub("[^A-Za-z0-9\s]","", text)
  
  # replace consecutive whitespace with single space
  text = re.sub('\s+', ' ', text)
  return text

def tokenization(cleaned_text:str)->list:
    cleaned_text = re.split('\W+', cleaned_text)
    cleaned_text = list(filter(None, cleaned_text)) #removes empty list element
    return cleaned_text

def remove_stopwords(tokenized_text:list)->list:

    stopwords = nltk.corpus.stopwords.words('english')
    stopwords.remove('no')# No and Not is important in sarcasm??
    stopwords.remove('not')

    stopwords = [remove_punctuations(word) for word in stopwords]
    tokenized_text = [word for word in tokenized_text if word.lower() not in stopwords]
    return tokenized_text

In [20]:
# newdf['tweet_cleaned'] = newdf['tweet'].apply(lambda x: clean_tweet(x))
# newdf['tweet_tokenized'] = newdf['tweet_cleaned'].apply(lambda x:tokenization(x))
# newdf['tweet_no_stopwords'] = newdf['tweet_tokenized'].apply(lambda x:remove_stopwords(x))

In [21]:
def print_statistics(y, y_pred):
    accuracy = metrics.accuracy_score(y, y_pred)
    precision = metrics.precision_score(y, y_pred, average='weighted')
    recall = metrics.recall_score(y, y_pred, average='weighted')
    f_score = metrics.f1_score(y, y_pred, average='weighted')
    print('Accuracy: %.3f\nPrecision: %.3f\nRecall: %.3f\nF_score: %.3f\n'
          % (accuracy, precision, recall, f_score))
    print(metrics.classification_report(y, y_pred))
    return accuracy, precision, recall, f_score



def plot_coefficients(classifier, feature_names, top_features=20, plot_name="/bow_models/bow_binary_"):
    # Get the top most positive/negative coefficients
    coef = classifier.coef_.ravel()
    top_positive_coefficients = np.argsort(coef)[-top_features:]
    top_negative_coefficients = np.argsort(coef)[:top_features]
    top_coefficients = np.hstack([top_negative_coefficients, top_positive_coefficients])
    x_names = [feature_names[feature] for feature in top_coefficients]

    # Plot the coefficients
    plt.figure(figsize=(15, 5))
    colors = ['red' if c < 0 else 'blue' for c in coef[top_coefficients]]
    plt.bar(np.arange(2 * top_features), coef[top_coefficients], color=colors)
    plt.xticks(np.arange(0, 2 * top_features), x_names, rotation=30, ha='right')
    plt.ylabel("Coefficient Value")
    plt.title("Visualising the top %d features taken up by an SVM model" % top_features)
    to_save_filename = path + "/plots/" + plot_name + "top%d_coefficients.png" % top_features
    plt.savefig(to_save_filename)
    print("Coefficients' visualisation saved to %s\n" % to_save_filename)

def get_regularization_params(a=-1, b=1, c=3, d=1, e=5):
    reg_range = np.outer(np.logspace(a, b, c), np.array([d, e]))
    reg_range = reg_range.flatten()
    return reg_range


def grid_classifier(x_train, y_train, x_test, y_test, model, parameters,
                    make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    grid = GridSearchCV(estimator=model, param_grid=parameters, verbose=0)
    grid.fit(x_train, y_train)
    sorted(grid.cv_results_.keys())
    classifier = grid.best_estimator_
    if make_feature_analysis:
        plot_coefficients(classifier, feature_names, top_features, plot_name)
    y_hat = classifier.predict(x_test)
    print_statistics(y_test, y_hat)

# Method to print the header of the currently running model
def print_model_title(name):
    print("\n==================================================================")
    print('{:>20}'.format(name))
    print("==================================================================\n")


def linear_svm_grid(x_train, y_train, x_test, y_test, class_ratio,
               make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Linear SVM")
    C_range = get_regularization_params()
    parameters = {'C': C_range}
    linear_svm = LinearSVC(C=1.0, class_weight=class_ratio, penalty='l2')
    grid_classifier(x_train, y_train, x_test, y_test, linear_svm, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def nonlinear_svm_grid(x_train, y_train, x_test, y_test, class_ratio,
                  make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Nonlinear SVM")
    C_range = get_regularization_params(a=-1, b=0, c=2, d=1, e=5)
    gamma_range = get_regularization_params(a=-2, b=-1, c=2, d=1, e=5)
    parameters = {'kernel': ['rbf'], 'C': C_range, 'gamma': gamma_range}
    nonlinear_svm = SVC(class_weight=class_ratio)
    grid_classifier(x_train, y_train, x_test, y_test, nonlinear_svm, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def logistic_regression_grid(x_train, y_train, x_test, y_test, class_ratio,
                        make_feature_analysis=False, feature_names=None, top_features=0, plot_name="coeff"):
    print_model_title("Logistic Regression")
    C_range = [0.001, 0.01, 0.1, 1, 10, 100]
    parameters = {'C': C_range}
    log_regr = LogisticRegression(C=1.0, class_weight=class_ratio, penalty='l2')
    grid_classifier(x_train, y_train, x_test, y_test, log_regr, parameters,
                    make_feature_analysis, feature_names, top_features, plot_name)


def linear_svm(x_train, y_train, x_test, y_test, class_ratio='balanced'):
    print_model_title("Linear SVM")
    svm = LinearSVC(C=0.01, class_weight=class_ratio, penalty='l2')
    svm.fit(x_train, y_train)
    y_hat = svm.predict(x_test)
    print_statistics(y_test, y_hat)
    # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/svm_model_bin.sav'
    # joblib.dump(svm, filename)


def logistic_regression(x_train, y_train, x_test, y_test, class_ratio='balanced'):
    print_model_title("Logistic Regression")
    regr = LogisticRegression(C=0.01, class_weight=class_ratio, penalty='l2')
    regr.fit(x_train, y_train)
    y_hat = regr.predict(x_test)
    print_statistics(y_test, y_hat)
    # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
    # joblib.dump(regr, filename)


def random_forest(x_train, y_train, x_test, y_test, class_ratio='balanced'):
  print_model_title("Random Forest")
  rf = RandomForestClassifier(n_estimators=400, random_state=11)
  rf.fit(x_train, y_train)
  y_hat = rf.predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)

def xg_boost(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost")
  xgb_model =XGBClassifier(max_depth=6, n_estimators=1000)
  xgb_model .fit(x_train, y_train)
  y_hat = xgb_model .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def xg_boost_focal_loss(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost Focal")
  xgboster_focal = imb_xgb(special_objective='focal')
  CV_focal_booster = GridSearchCV(xgboster_focal, {"focal_gamma":[1.0,1.5,2.0,2.5,3.0]})
  CV_focal_booster.fit(x_train, y_train)
  opt_focal_booster = CV_focal_booster.best_estimator_
  # xgb_model .fit(x_train, y_train)
  y_hat = opt_focal_booster.predict_determine(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)

def xg_boost_weighted_loss(x_train, y_train, x_test, y_test):
  print_model_title("XGBoost Weighted")
  xgboster_focal = imb_xgb(special_objective='weighted')
  CV_focal_booster = GridSearchCV(xgboster_focal, {"imbalance_alpha":[1.5,2.0,2.5,3.0,4.0]})
  CV_focal_booster.fit(x_train, y_train)
  opt_focal_booster = CV_focal_booster.best_estimator_
  # xgb_model .fit(x_train, y_train)
  y_hat = opt_focal_booster.predict_determine(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def feature_selection(x_train, y_train, x_test, y_test):
    print("Feature selection with LinearSVC")
    model = LinearSVC(C=0.1, penalty='l2')
    rfe = RFE(model, 5)
    best_features_model = rfe.fit(x_train, y_train)
    y_hat = best_features_model.predict(x_test)
    print_statistics(y_test, y_hat)


def ensemble_stacked(x_train, y_train, x_test, y_test):
  print_model_title("Ensemble Stacked Classifiers")
  estimators = [ ('lr',LogisticRegression(C=0.01, class_weight='balanced', penalty='l2')),('xgb',XGBClassifier(max_depth=16, n_estimators=1000)),('svm_linear',LinearSVC(C=0.01, class_weight='balanced', penalty='l2')),('rf', RandomForestClassifier(n_estimators=10, random_state=42))]
  from sklearn.ensemble import StackingClassifier
  clf = StackingClassifier(
      estimators=estimators )
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)



from sklearn.ensemble import RandomForestClassifier, VotingClassifier
def voting_classifiers(x_train, y_train, x_test, y_test,voting_type='hard'):
  print_model_title("Voting Classifier")
  estimators = [ ('lr',LogisticRegression(C=0.01, class_weight='balanced', penalty='l2')),('xgb',XGBClassifier(max_depth=16, n_estimators=1000)),('svm_linear',LinearSVC(C=0.01, class_weight='balanced', penalty='l2')),('rf', RandomForestClassifier(n_estimators=10, random_state=42))]
  from sklearn.ensemble import StackingClassifier
  clf = VotingClassifier(
      estimators=estimators , voting=voting_type)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)



from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import BaggingClassifier
def Bagging_Classifier_LR(x_train, y_train, x_test, y_test):
  print_model_title("Bagging Calssifier LR")
 
  clf =BaggingClassifier(base_estimator=LogisticRegression(C=0.01, class_weight='balanced', penalty='l2'),
                       n_estimators=10, random_state=42)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


def Bagging_Classifier_SVM(x_train, y_train, x_test, y_test):
  print_model_title("Bagging Calssifier SVM")
 
  clf =BaggingClassifier(base_estimator=LinearSVC(C=0.01, class_weight='balanced', penalty='l2'),
                       n_estimators=10, random_state=42)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)
  # filename = '/content/drive/MyDrive/Omdena_dialect_identification/Saved_models/lr_model_bin.sav'
  # joblib.dump(regr, filename)


from sklearn.ensemble import GradientBoostingClassifier
def gradient_boosting(x_train, y_train, x_test, y_test):
  print_model_title("Gradient Boosting")
 
  clf =GradientBoostingClassifier(n_estimators=100, learning_rate=0.01,max_depth=30, random_state=42)
  clf.fit(x_train, y_train)
  y_hat = clf .predict(x_test)
  print_statistics(y_test, y_hat)


In [213]:
newdf.dropna(inplace=True)

In [214]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    ngram_range=(1,1),
    dtype=np.float32,
    max_features=5000
)


# Character Stemmer
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    ngram_range=(1, 4),
    dtype=np.float32,
    max_features=4000
)

word_vectorizer.fit(newdf['tweet'].values)
char_vectorizer.fit(newdf['tweet'].values)


TfidfVectorizer(analyzer='char', dtype=<class 'numpy.float32'>,
                max_features=4000, ngram_range=(1, 4), strip_accents='unicode',
                sublinear_tf=True)

In [24]:
import warnings
warnings.filterwarnings('ignore')

In [160]:
y_train=newdf['sarcastic'].values

In [161]:
y_train

array([0, 0, 0, ..., 1, 1, 1])

In [215]:
y_test=test['sarcastic'].values

In [216]:
train_word_features = word_vectorizer.transform(newdf['tweet'].values)
train_char_features = char_vectorizer.transform(newdf['tweet'].values)

In [217]:
test.dropna(inplace=True)

In [218]:
test_word_features = word_vectorizer.transform(test['tweet'].values)
test_char_features = char_vectorizer.transform(test['tweet'].values)

In [165]:
train_features = hstack([
    train_char_features,
    train_word_features]
    , 'csr'
)

In [166]:
test_features = hstack([
    test_char_features,
    test_word_features]
    , 'csr'
)

In [38]:
linear_svm_grid(train_features,y_train,test_features,y_test,class_ratio='balanced')


          Linear SVM

Accuracy: 0.726
Precision: 0.714
Recall: 0.726
F_score: 0.719

              precision    recall  f1-score   support

           0       0.81      0.84      0.82       531
           1       0.40      0.35      0.38       162

    accuracy                           0.73       693
   macro avg       0.61      0.60      0.60       693
weighted avg       0.71      0.73      0.72       693



In [40]:
logistic_regression_grid(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


 Logistic Regression

Accuracy: 0.717
Precision: 0.707
Recall: 0.717
F_score: 0.712

              precision    recall  f1-score   support

           0       0.81      0.83      0.82       531
           1       0.38      0.35      0.36       162

    accuracy                           0.72       693
   macro avg       0.59      0.59      0.59       693
weighted avg       0.71      0.72      0.71       693



In [41]:
linear_svm(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


          Linear SVM

Accuracy: 0.661
Precision: 0.693
Recall: 0.661
F_score: 0.675

              precision    recall  f1-score   support

           0       0.81      0.73      0.77       531
           1       0.33      0.42      0.37       162

    accuracy                           0.66       693
   macro avg       0.57      0.58      0.57       693
weighted avg       0.69      0.66      0.67       693



In [42]:
logistic_regression(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


 Logistic Regression

Accuracy: 0.635
Precision: 0.700
Recall: 0.635
F_score: 0.658

              precision    recall  f1-score   support

           0       0.82      0.68      0.74       531
           1       0.32      0.50      0.39       162

    accuracy                           0.63       693
   macro avg       0.57      0.59      0.56       693
weighted avg       0.70      0.63      0.66       693



In [43]:
random_forest(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')



       Random Forest

Accuracy: 0.687
Precision: 0.662
Recall: 0.687
F_score: 0.673

              precision    recall  f1-score   support

           0       0.78      0.83      0.80       531
           1       0.28      0.22      0.25       162

    accuracy                           0.69       693
   macro avg       0.53      0.53      0.53       693
weighted avg       0.66      0.69      0.67       693



In [44]:
xg_boost(train_features,y_train.astype(int),test_features,y_test.astype(int))



             XGBoost

Accuracy: 0.701
Precision: 0.671
Recall: 0.701
F_score: 0.683

              precision    recall  f1-score   support

           0       0.78      0.85      0.81       531
           1       0.31      0.22      0.26       162

    accuracy                           0.70       693
   macro avg       0.54      0.53      0.54       693
weighted avg       0.67      0.70      0.68       693



In [47]:
Bagging_Classifier_LR(train_features,y_train.astype(int),test_features,y_test.astype(int))



Bagging Calssifier LR

Accuracy: 0.625
Precision: 0.696
Recall: 0.625
F_score: 0.649

              precision    recall  f1-score   support

           0       0.81      0.66      0.73       531
           1       0.31      0.50      0.38       162

    accuracy                           0.62       693
   macro avg       0.56      0.58      0.56       693
weighted avg       0.70      0.62      0.65       693



In [48]:
Bagging_Classifier_SVM(train_features,y_train.astype(int),test_features,y_test.astype(int))



Bagging Calssifier SVM

Accuracy: 0.664
Precision: 0.693
Recall: 0.664
F_score: 0.676

              precision    recall  f1-score   support

           0       0.81      0.74      0.77       531
           1       0.33      0.41      0.37       162

    accuracy                           0.66       693
   macro avg       0.57      0.58      0.57       693
weighted avg       0.69      0.66      0.68       693



In [50]:
def count_character_type(text):
    text=str(text)

    specialChar = 0
    for i in range(0, len(text)):
        ch = text[i]
        if ch == "!":
            specialChar+=1
        
    return specialChar
    
def count_question_mark(text):
    text=str(text)
    specialChar = 0
    for i in range(0, len(text)):
        ch = text[i]
        if ch == "?":
            specialChar+=1
        
    return specialChar

The Python NLP library, TextBlob has a method to roughly quantify if a sentence is fact or opinion.¶
The method outputs a number ranging from 0 to 1, an output close to 0 indicates the sentence is highly factual and close to 1 means the sentence is highly subjective. Here, we take the sum for every comment, an overall higher sum then, will be indicative of higher subjectivity

In [51]:
def get_subjectivity(text):
    return TextBlob(str(text)).sentiment.subjectivity

The distribution of the 4 most important parts-of-speech (Noun, Verb, Adjective,Pronoun) in sarcastic and non sarcastic comments might shed some light, so let's plot it as a grouped bar chart

In [52]:
import spacy

In [53]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [54]:
nlp = spacy.load('en_core_web_sm')
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [55]:
def pos_counter(doc):
    verb_count = 0
    adj_count = 0
    pron_count = 0
    noun_count=0
    for tok in doc:
        if tok.pos_ == "VERB":
            verb_count=verb_count+1
        elif tok.pos_ == "ADJ":
            adj_count=adj_count+1
        elif tok.pos_ == "PRON":
            pron_count=pron_count+1
        elif tok.pos_ == "PROPN":
            noun_count=noun_count+1
    return (verb_count,adj_count,pron_count,noun_count)

In [56]:
def get_verb_count(text):
  verb_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "VERB":
            verb_count=verb_count+1
  return verb_count

def get_noun_count(text):
  noun_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "PROPN":
            noun_count=noun_count+1
  return noun_count

def get_pron_count(text):
  pron_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "PRON":
            pron_count=pron_count+1
  return pron_count


def get_adj_count(text):
  adj_count = 0
  doc = nlp(str(text))
  for tok in doc:
        if tok.pos_ == "ADJ":
            adj_count=adj_count+1
  return adj_count

Sarcastic comments tend to have profanity interspersed within, let's get a count of both sarcastic and non sarcastic and see how they compare

In [57]:
!pip install better_profanity 
from better_profanity import profanity

     |████████████████████████████████| 46 kB 1.8 MB/s 


In [58]:
def count_profane_words(text):
    count = 0
    for sent in str(text).split():
        if profanity.contains_profanity(sent) == True:
            count = count+1
    return count

get entity cloud

In [59]:
def entity_wordcloud(text):
    named_entities = []    
    sent = nlp(str(text))
    for ent in sent.ents:
        if ent.label_ == 'PERSON' or 'ORG' or 'GPE':
            named_entities.append(ent.text)
            
    doc = " ".join(named_entities)
    return doc

In [60]:
!git clone https://github.com/larsmans/sentiwordnet.git

Cloning into 'sentiwordnet'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6 (delta 0), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [61]:
import csv

In [62]:
class load_senti_word_net(object):
    """
    constructor to load the file and read the file as CSV
    6 columns - pos, ID, PosScore, NegScore, synsetTerms, gloss
    synsetTerms can have multiple similar words like abducting#1 abducent#1 and will read each one and calculaye the scores
    """
    
    def __init__(self):
        sent_scores = collections.defaultdict(list)
        with io.open("/content/sentiwordnet/SentiWordNet_3.0.0_20130122.txt") as fname:
            file_content = csv.reader(fname, delimiter='\t',quotechar='"')
            
            for line in file_content:                
                if line[0].startswith('#') :
                    continue                    
                pos, ID, PosScore, NegScore, synsetTerms, gloss = line
                for terms in synsetTerms.split(" "):
                    term = terms.split("#")[0]
                    term = term.replace("-","").replace("_","")
                    key = "%s/%s"%(pos,term.split("#")[0])
                    try:
                        sent_scores[key].append((float(PosScore),float(NegScore)))
                    except:
                        sent_scores[key].append((0,0))
                    
        for key, value in sent_scores.items():
            sent_scores[key] = np.mean(value,axis=0)
        
        self.sent_scores = sent_scores    
     
    """
    For a word,
    nltk.pos_tag(["Suraj"])
    [('Suraj', 'NN')]
    """
    
    def score_word(self, word):
        pos = nltk.pos_tag([word])[0][1]
        return self.score(word, pos)
    
    def score(self,word, pos):
        """
        Identify the type of POS, get the score from the senti_scores and return the score
        """
        
        if pos[0:2] == 'NN':
            pos_type = 'n'
        elif pos[0:2] == 'JJ':
            pos_type = 'a'
        elif pos[0:2] =='VB':
            pos_type='v'
        elif pos[0:2] =='RB':
            pos_type = 'r'
        else:
            pos_type =  0
            
        if pos_type != 0 :    
            loc = pos_type+'/'+word
            score = self.sent_scores[loc]
            if len(score)>1:
                return score
            else:
                return np.array([0.0,0.0])
        else:
            return np.array([0.0,0.0])
        
    """
    Repeat the same for a sentence
    nltk.pos_tag(word_tokenize("My name is Suraj"))
    [('My', 'PRP$'), ('name', 'NN'), ('is', 'VBZ'), ('Suraj', 'NNP')]    
    """    
        
    def score_sentencce(self, sentence):
        pos = nltk.pos_tag(sentence)
        # print (pos)
        mean_score = np.array([0.0, 0.0])
        for i in range(len(pos)):
            mean_score += self.score(pos[i][0], pos[i][1])
            
        return mean_score
    
    def pos_vector(self, sentence):
        pos_tag = nltk.pos_tag(sentence)
        vector = np.zeros(4)
        
        for i in range(0, len(pos_tag)):
            pos = pos_tag[i][1]
            if pos[0:2]=='NN':
                vector[0] += 1
            elif pos[0:2] =='JJ':
                vector[1] += 1
            elif pos[0:2] =='VB':
                vector[2] += 1
            elif pos[0:2] == 'RB':
                vector[3] += 1
                
        return vector
            

In [63]:
def gram_features(features,sentence):
    sentence_rep = replace_reg(str(sentence))
    token = nltk.word_tokenize(sentence_rep)
    token = [porter.stem(i.lower()) for i in token]        
    
    bigrams = nltk.bigrams(token)
    bigrams = [tup[0] + ' ' + tup[1] for tup in bigrams]
    grams = token + bigrams
    #print (grams)
    for t in grams:
        features['contains(%s)'%t]=1.0

In [64]:
def pos_features(features,sentence):
    sentence_rep = replace_reg(sentence)
    token = nltk.word_tokenize(sentence_rep)
    token = [ porter.stem(each.lower()) for each in token]
    pos_vector = sentiments.pos_vector(token)
    for j in range(len(pos_vector)):
        features['POS_'+str(j+1)] = pos_vector[j]
    # print ("done")
    

In [65]:
def capitalization(features,sentence):
    count = 0
    for i in range(len(sentence)):
        count += int(sentence[i].isupper())
    features['Capitalization'] = int(count > 3)
    # print (count)

In [66]:
import nltk
import re

#dictionnary to sentiment analysis
emo_repl = {
    #good emotions
    "&lt;3" : " good ",
    ":d" : " good ",
    ":dd" : " good ",
    ":p" : " good ",
    "8)" : " good ",
    ":-)" : " good ",
    ":)" : " good ",
    ";)" : " good ",
    "(-:" : " good ",
    "(:" : " good ",
    
    "yay!" : " good ",
    "yay" : " good ",
    "yaay" : " good ",
    "yaaay" : " good ",
    "yaaaay" : " good ",
    "yaaaaay" : " good ",    
    #bad emotions
    ":/" : " bad ",
    ":&gt;" : " sad ",
    ":')" : " sad ",
    ":-(" : " bad ",
    ":(" : " bad ",
    ":s" : " bad ",
    ":-s" : " bad "
}

#dictionnary for general (i.e. topic modeler)
emo_repl2 = {
    #good emotions
    "&lt;3" : " heart ",
    ":d" : " smile ",
    ":p" : " smile ",
    ":dd" : " smile ",
    "8)" : " smile ",
    ":-)" : " smile ",
    ":)" : " smile ",
    ";)" : " smile ",
    "(-:" : " smile ",
    "(:" : " smile ",
       
    #bad emotions
    ":/" : " worry ",
    ":&gt;" : " angry ",
    ":')" : " sad ",
    ":-(" : " sad ",
    ":(" : " sad ",
    ":s" : " sad ",
    ":-s" : " sad "
}

#general
re_repl = {
    r"\br\b" : "are",
    r"\bu\b" : "you",
    r"\bhaha\b" : "ha",
    r"\bhahaha\b" : "ha",
    r"\bdon't\b" : "do not",
    r"\bdoesn't\b" : "does not",
    r"\bdidn't\b" : "did not",
    r"\bhasn't\b" : "has not",
    r"\bhaven't\b" : "have not",
    r"\bhadn't\b" : "had not",
    r"\bwon't\b" : "will not",
    r"\bwouldn't\b" : "would not",
    r"\bcan't\b" : "can not",
    r"\bcannot\b" : "can not"    
}

emo_repl_order = [k for (k_len,k) in reversed(sorted([(len(k),k) for k in emo_repl.keys()]))]
emo_repl_order2 = [k for (k_len,k) in reversed(sorted([(len(k),k) for k in emo_repl2.keys()]))]

def replace_emo(sentence):
    sentence2 = sentence
    for k in emo_repl_order:
        sentence2 = sentence2.replace(k,emo_repl[k])
    for r, repl in re_repl.items():
        sentence2 = re.sub(r,repl,sentence2)
    return sentence2

def replace_reg(sentence):
    sentence2 = sentence
    for k in emo_repl_order2:
        sentence2 = sentence2.replace(k,emo_repl2[k])
    for r, repl in re_repl.items(): 
        sentence2 = re.sub(r,repl,sentence2)
    return sentence2

In [67]:
from gensim import corpora, models, similarities
import nltk
from nltk.corpus import stopwords
import numpy as np
import pandas as pd

class topic(object):
    def __init__(self, nbtopic = 100, alpha=1,model=None,dicttp=None):
        self.nbtopic = nbtopic
        self.alpha = alpha
        self.porter = nltk.PorterStemmer()
        self.stop = stopwords.words('english')+['.','!','?','"','...','\\',"''",'[',']','~',"'m","'s",';',':','..','$']
        if model!=None and dicttp!=None:
            self.lda = models.ldamodel.LdaModel.load(model)
            self.dictionary =  corpora.Dictionary.load(dicttp)
            
    def fit(self,documents):
        
        documents_mod = documents
        tokens = [nltk.word_tokenize(sentence) for sentence in documents_mod]
        tokens = [[self.porter.stem(t.lower()) for t in sentence if t.lower() not in self.stop] for sentence in tokens]        
            
        self.dictionary = corpora.Dictionary(tokens)
        corpus = [self.dictionary.doc2bow(text) for text in tokens]
        self.lda = models.ldamodel.LdaModel(corpus,id2word=self.dictionary, num_topics=self.nbtopic,alpha=self.alpha)
        
        self.lda.save('topics.tp')
        self.dictionary.save('topics_dict.tp')
        
    def get_topic(self,topic_number):
        
        return self.lda.print_topic(topic_number)
    
    def transform(self,sentence):
        
        sentence_mod = sentence
        tokens = nltk.word_tokenize(sentence_mod)
        tokens = [self.porter.stem(t.lower()) for t in tokens if t.lower() not in self.stop] 
        corpus_sentence = self.dictionary.doc2bow(tokens)
        
        return self.lda[corpus_sentence] 

In [68]:
porter = nltk.PorterStemmer()


In [69]:
import string
def sentiment_extract(features, sentence):
    sentence_rep = replace_reg(sentence)
    token = nltk.word_tokenize(sentence_rep)    
    token = [porter.stem(i.lower()) for i in token]   
    mean_sentiment = sentiments.score_sentencce(token)
    features["Positive Sentiment"] = mean_sentiment[0]
    features["Negative Sentiment"] = mean_sentiment[1]
    features["sentiment"] = mean_sentiment[0] - mean_sentiment[1]
    #print(mean_sentiment[0], mean_sentiment[1])
    
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in token]).strip())
        features["Blob Polarity"] = text.sentiment.polarity
        features["Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["Blob Polarity"] = 0
        features["Blob Subjectivity"] = 0
        # print("do nothing")
        
    
    first_half = token[0:len(token)//2]    
    mean_sentiment_half = sentiments.score_sentencce(first_half)
    features["positive Sentiment first half"] = mean_sentiment_half[0]
    features["negative Sentiment first half"] = mean_sentiment_half[1]
    features["first half sentiment"] = mean_sentiment_half[0]-mean_sentiment_half[1]
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in first_half]).strip())
        features["first half Blob Polarity"] = text.sentiment.polarity
        features["first half Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["first Blob Polarity"] = 0
        features["first Blob Subjectivity"] = 0
        # print("do nothing")
    
    second_half = token[len(token)//2:]
    mean_sentiment_sechalf = sentiments.score_sentencce(second_half)
    features["positive Sentiment second half"] = mean_sentiment_sechalf[0]
    features["negative Sentiment second half"] = mean_sentiment_sechalf[1]
    features["second half sentiment"] = mean_sentiment_sechalf[0]-mean_sentiment_sechalf[1]
    try:
        text = TextBlob(" ".join([""+i if i not in string.punctuation and not i.startswith("'") else i for i in second_half]).strip())
        features["second half Blob Polarity"] = text.sentiment.polarity
        features["second half Blob Subjectivity"] = text.sentiment.subjectivity
        #print (text.sentiment.polarity,text.sentiment.subjectivity )
    except:
        features["second Blob Polarity"] = 0
        features["second Blob Subjectivity"] = 0
        # print("do nothing")  
    

In [70]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [71]:
topic_mod = topic(nbtopic=200,alpha='symmetric')
topic_mod.fit(newdf['tweet'].values)
# topic_mod = topic(model=os.path.join('topics.tp'),dicttp=os.path.join('topics_dict.tp'))
def topic_feature(features,sentence,topic_modeler):    
    topics = topic_modeler.transform(sentence)    
    for j in range(len(topics)):
        features['Topic :'] = topics[j][1]

In [72]:
!pip install emoji

     |████████████████████████████████| 170 kB 5.5 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=2ee34b1bf81017b9f6f99bb7ee79a3a048aa5531c062ee64acc9daf53311b4fe
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [73]:
from emoji import UNICODE_EMOJI



In [74]:
"""
Function to extract the below features.
1. Presence of emoji's
2. Count of number of Question marks
3. Count of number of Exclamation marks
4. presence of hashtags other than #sarcasm
5. presence of any @user tweets

"""
from collections import defaultdict

def Emoji_present(text):
    emoji = {
    "&lt;3" : "positive",":D" : "positive",	":d" : "positive", ":dd" : "positive", ":P" : "positive", ":p" : "positive","8)" : "positive",
    "8-)" : "positive",  ":-)" : "positive",    ":)" : "positive",    ";)" : "positive",    "(-:" : "positive",    "(:" : "positive",
    ":')" : "positive",    "xD" : "positive",    "XD" : "positive",  "yay!" : "positive",  "yay" : "positive",  "yaay" : "positive",
    "yaaay" : "positive",  "yaaaay" : "positive", "yaaaaay" : "positive", "Yay!" : "positive", "Yay" : "positive", "Yaay" : "positive",
    "Yaaay" : "positive", "Yaaaay" : "positive", "Yaaaaay" : "positive",  ":/" : "negative", "&gt;" : "negative", ":'(" : "negative",
    ":-(" : "negative", ":(" : "negative", ":s" : "negative",":-s" : "negative","-_-" : "negative", "-.-" : "negative" }
    emoji_count =defaultdict(int) 
    isPresent=False    
        
    for word in str(text):
        if word in emoji:            
            isPresent = True

    
    count = 0
    for emoji in UNICODE_EMOJI:
        count += str(text).count(emoji)
        if count>=1:
          isPresent=True      

    
    return isPresent 



In [75]:
import io

sentiments = load_senti_word_net()

def get_features(sentence, topic_modeler):
    features = {}
    # gram_features(features,sentence)
    pos_features(features,sentence)
    sentiment_extract(features, sentence)
    capitalization(features,sentence)
    topic_feature(features, sentence,topic_modeler)
    return features

In [76]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [77]:
newdf.dropna(inplace=True)

In [78]:
newdf

,tweet,sarcastic
0,Less than 5 minutes after getting my phone bac...,0
4,Every time I comment on a TikTok asking for ca...,0
6,@mikk2k2 @ArgoBlockchain All/any updates are f...,0
7,I love my writing teacher sm plsss,0
10,there isn’t a labor shortage. no one wants to ...,0
...,...,...
1,I've never preserved sex,1
0,take my looool mmmm 9k xxxxx to have to be phy...,1
1,"""Tat, butter for fat, you kill my dog, I eat y...",1
0,"I'm so glad the clocks just went back, I have ...",1


In [167]:
text__=newdf['tweet'].values
label=newdf['sarcastic'].values

In [168]:
import re
featureset=[]
for i in range(0,len(text__)):
    # if i==2774or i==2775:
    #   continue
    temp = str(text__[i])
    temp = re.sub(r'[^\x00-\x7F]+','',temp)
    featureset.append((get_features(temp,topic_mod), label[i], text__[i]))

In [219]:
c = []
for i in range(0,len(featureset)):
    c.append(pd.DataFrame(featureset[i][0],index=[i]))
result = pd.concat(c)

In [220]:
result

,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :
0,11.0,2.0,2.0,1.0,0.095620,0.432559,-0.336939,-0.083333,0.033333,0.076389,0.315972,-0.239583,-0.083333,0.033333,0.019231,0.116587,-0.097356,0.00,0.000000,0,0.048614
1,15.0,3.0,1.0,0.0,0.717014,0.522222,0.194792,0.000000,0.100000,0.458333,0.466667,-0.008333,0.000000,0.100000,0.258681,0.055556,0.203125,0.00,0.000000,1,0.056229
2,8.0,2.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,1,0.063472
3,4.0,1.0,1.0,0.0,0.687500,0.031250,0.656250,0.500000,0.600000,0.625000,0.031250,0.593750,0.500000,0.600000,0.062500,0.000000,0.062500,0.00,0.000000,0,0.179454
4,15.0,3.0,6.0,1.0,0.755978,1.302066,-0.546088,0.400000,0.800000,0.374033,0.363871,0.010162,0.000000,0.000000,0.381944,0.938194,-0.556250,0.40,0.800000,0,0.649999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4133,2.0,0.0,2.0,1.0,0.062500,0.500000,-0.437500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.500000,-0.437500,0.00,0.000000,0,0.301387
4134,5.0,1.0,3.0,1.0,0.337273,0.101624,0.235649,0.000000,0.000000,0.058427,0.082393,-0.023966,0.000000,0.000000,0.278846,0.019231,0.259615,0.00,0.000000,0,0.286429
4135,10.0,2.0,4.0,0.0,0.833333,1.096577,-0.263244,0.000000,0.000000,0.308333,0.996577,-0.688244,0.000000,0.000000,0.525000,0.100000,0.425000,0.00,0.000000,1,0.214850
4136,5.0,3.0,5.0,3.0,0.806939,1.313941,-0.507002,0.175000,0.375000,0.572917,0.312500,0.260417,0.250000,0.500000,0.234023,1.001441,-0.767419,0.10,0.250000,0,0.100752


In [221]:
result.insert(loc=0,column="label",value='0')
for i in range(0, len(featureset)):
    result["label"].loc[i] = featureset[i][1]   

result.insert(loc=0,column="text",value='0')
for i in range(0, len(featureset)):
    result["text"].loc[i] = featureset[i][2]  
    

In [222]:
result.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :
0,Less than 5 minutes after getting my phone bac...,0,11.0,2.0,2.0,1.0,0.095620,0.432559,-0.336939,-0.083333,0.033333,0.076389,0.315972,-0.239583,-0.083333,0.033333,0.019231,0.116587,-0.097356,0.0,0.0,0,0.048614
1,Every time I comment on a TikTok asking for ca...,0,15.0,3.0,1.0,0.0,0.717014,0.522222,0.194792,0.000000,0.100000,0.458333,0.466667,-0.008333,0.000000,0.100000,0.258681,0.055556,0.203125,0.0,0.0,1,0.056229
2,@mikk2k2 @ArgoBlockchain All/any updates are f...,0,8.0,2.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1,0.063472
3,I love my writing teacher sm plsss,0,4.0,1.0,1.0,0.0,0.687500,0.031250,0.656250,0.500000,0.600000,0.625000,0.031250,0.593750,0.500000,0.600000,0.062500,0.000000,0.062500,0.0,0.0,0,0.179454
4,there isn’t a labor shortage. no one wants to ...,0,15.0,3.0,6.0,1.0,0.755978,1.302066,-0.546088,0.400000,0.800000,0.374033,0.363871,0.010162,0.000000,0.000000,0.381944,0.938194,-0.556250,0.4,0.8,0,0.649999


In [223]:
result['contain_emoji']=result['text'].apply(Emoji_present)

In [224]:
result['count_special_chracter']=result['text'].apply(count_character_type)

In [225]:
result['count_question_marks']=result['text'].apply(count_question_mark)

In [226]:
result['subjectivity']=result['text'].apply(get_subjectivity)

In [227]:
result['count_verbs']=result['text'].apply(get_verb_count)

In [228]:
result['count_nouns']=result['text'].apply(get_noun_count)

In [229]:
result['count_pronun']=result['text'].apply(get_pron_count)

In [230]:
result['count_adjct']=result['text'].apply(get_adj_count)

In [231]:
result['entities']=result['text'].apply(entity_wordcloud)

In [232]:
result['count_profane']=result['text'].apply(count_profane_words)

In [183]:
result.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :,contain_emoji,count_special_chracter,count_question_marks,subjectivity,count_verbs,count_nouns,count_pronun,count_adjct,entities,count_profane
0,Less than 5 minutes after getting my phone bac...,0,11.0,2.0,2.0,1.0,0.095620,0.432559,-0.336939,-0.083333,0.033333,0.076389,0.315972,-0.239583,-0.083333,0.033333,0.019231,0.116587,-0.097356,0.0,0.0,0,0.048614,True,0,0,0.033333,2,0,1,1,Less than 5 minutes,0
1,Every time I comment on a TikTok asking for ca...,0,15.0,3.0,1.0,0.0,0.717014,0.522222,0.194792,0.000000,0.100000,0.458333,0.466667,-0.008333,0.000000,0.100000,0.258681,0.055556,0.203125,0.0,0.0,1,0.056229,True,0,0,0.533333,6,4,3,2,TikTok,0
2,@mikk2k2 @ArgoBlockchain All/any updates are f...,0,8.0,2.0,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1,0.063472,True,0,0,0.266667,1,1,0,1,Mike,0
3,I love my writing teacher sm plsss,0,4.0,1.0,1.0,0.0,0.687500,0.031250,0.656250,0.500000,0.600000,0.625000,0.031250,0.593750,0.500000,0.600000,0.062500,0.000000,0.062500,0.0,0.0,0,0.179454,True,0,0,0.600000,1,0,1,0,,0
4,there isn’t a labor shortage. no one wants to ...,0,15.0,3.0,6.0,1.0,0.755978,1.302066,-0.546088,0.400000,0.800000,0.374033,0.363871,0.010162,0.000000,0.000000,0.381944,0.938194,-0.556250,0.4,0.8,0,0.649999,True,0,0,0.450000,4,3,2,2,hogs &amp under 30,0


In [184]:
len(test)

693

In [185]:
text__test=test['tweet'].values
label_test=test['sarcastic'].values

In [186]:
import re
feature_set_test=[]
for i in range (0,len(text__test)):
    # if i==0:
    #   continue
    temp = str(text__test[i])
    temp = re.sub(r'[^\x00-\x7F]+','',temp)
    feature_set_test.append((get_features(temp,topic_mod), label_test[i],text__test[i]))

In [187]:
c_test = []
for i in range(0,len(feature_set_test)):
    c_test.append(pd.DataFrame(feature_set_test[i][0],index=[i]))
result_test = pd.concat(c_test)

In [188]:
result_test.insert(loc=0,column="label",value='0')
for i in range(0, len(feature_set_test)):
    result_test["label"].loc[i] = feature_set_test[i][1]   

result_test.insert(loc=0,column="text",value='0')
for i in range(0, len(feature_set_test)):
    result_test["text"].loc[i] = feature_set_test[i][2]   
    
    

In [189]:
result_test.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :
0,The world according to Jeff Goldblum is such a...,0,4.0,1.0,2.0,0.0,0.322917,0.354167,-0.031250,0.000000,0.500000,0.156250,0.187500,-0.031250,0.00,0.0,0.166667,0.166667,0.000000,0.000000,0.500000,0,0.251250
1,@kevinabstract ye,0,2.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.751250
2,Rubbing my tears into my face is part of my sk...,1,6.0,0.0,2.0,0.0,0.019231,0.184295,-0.165064,0.800000,0.700000,0.019231,0.184295,-0.165064,0.00,0.0,0.000000,0.000000,0.000000,0.800000,0.700000,0,0.167963
3,What's the German word for when you start a po...,0,10.0,5.0,7.0,3.0,0.922618,0.699750,0.222868,0.134091,0.188636,0.216518,0.158482,0.058036,0.05,0.1,0.706100,0.541268,0.164833,0.218182,0.277273,0,0.031125
4,"Prayers go up, blessings come down",0,2.0,0.0,2.0,0.0,0.077381,0.010119,0.067262,-0.155556,0.288889,0.041667,0.004167,0.037500,0.00,0.0,0.035714,0.005952,0.029762,-0.155556,0.288889,0,0.383204


In [190]:
result_test['contain_emoji']=result_test['text'].apply(Emoji_present)

In [191]:
result_test['count_special_chracter']=result_test['text'].apply(count_character_type)

In [192]:
result_test['count_question_marks']=result_test['text'].apply(count_question_mark)

In [193]:
result_test['subjectivity']=result_test['text'].apply(get_subjectivity)

In [194]:
result_test['count_verbs']=result_test['text'].apply(get_verb_count)

In [195]:
result_test['count_nouns']=result_test['text'].apply(get_noun_count)

In [196]:
result_test['count_pronun']=result_test['text'].apply(get_pron_count)

In [197]:
result_test['count_adjct']=result_test['text'].apply(get_adj_count)

In [198]:
result_test['entities']=result_test['text'].apply(entity_wordcloud)

In [199]:
result_test['count_profane']=result_test['text'].apply(count_profane_words)

In [200]:
result_test.head()

,text,label,POS_1,POS_2,POS_3,POS_4,Positive Sentiment,Negative Sentiment,sentiment,Blob Polarity,Blob Subjectivity,positive Sentiment first half,negative Sentiment first half,first half sentiment,first half Blob Polarity,first half Blob Subjectivity,positive Sentiment second half,negative Sentiment second half,second half sentiment,second half Blob Polarity,second half Blob Subjectivity,Capitalization,Topic :,contain_emoji,count_special_chracter,count_question_marks,subjectivity,count_verbs,count_nouns,count_pronun,count_adjct,entities,count_profane
0,The world according to Jeff Goldblum is such a...,0,4.0,1.0,2.0,0.0,0.322917,0.354167,-0.031250,0.000000,0.500000,0.156250,0.187500,-0.031250,0.00,0.0,0.166667,0.166667,0.000000,0.000000,0.500000,0,0.251250,False,0,0,0.500000,1,2,0,0,Jeff Goldblum,0
1,@kevinabstract ye,0,2.0,1.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.751250,False,0,0,0.000000,0,2,0,0,,0
2,Rubbing my tears into my face is part of my sk...,1,6.0,0.0,2.0,0.0,0.019231,0.184295,-0.165064,0.800000,0.700000,0.019231,0.184295,-0.165064,0.00,0.0,0.000000,0.000000,0.000000,0.800000,0.700000,0,0.167963,False,0,0,0.700000,1,0,0,0,,0
3,What's the German word for when you start a po...,0,10.0,5.0,7.0,3.0,0.922618,0.699750,0.222868,0.134091,0.188636,0.216518,0.158482,0.058036,0.05,0.1,0.706100,0.541268,0.164833,0.218182,0.277273,0,0.031125,True,0,1,0.188636,4,0,5,4,German 2 years old present-day every week 3 mo...,0
4,"Prayers go up, blessings come down",0,2.0,0.0,2.0,0.0,0.077381,0.010119,0.067262,-0.155556,0.288889,0.041667,0.004167,0.037500,0.00,0.0,0.035714,0.005952,0.029762,-0.155556,0.288889,0,0.383204,True,0,0,0.288889,2,0,0,0,,0


In [201]:
result_test.columns

Index(['text', 'label', 'POS_1', 'POS_2', 'POS_3', 'POS_4',
       'Positive Sentiment', 'Negative Sentiment', 'sentiment',
       'Blob Polarity', 'Blob Subjectivity', 'positive Sentiment first half',
       'negative Sentiment first half', 'first half sentiment',
       'first half Blob Polarity', 'first half Blob Subjectivity',
       'positive Sentiment second half', 'negative Sentiment second half',
       'second half sentiment', 'second half Blob Polarity',
       'second half Blob Subjectivity', 'Capitalization', 'Topic :',
       'contain_emoji', 'count_special_chracter', 'count_question_marks',
       'subjectivity', 'count_verbs', 'count_nouns', 'count_pronun',
       'count_adjct', 'entities', 'count_profane'],
      dtype='object')

In [235]:
x_train.isna().any()

POS_1                             False
POS_2                             False
POS_3                             False
POS_4                             False
Positive Sentiment                False
Negative Sentiment                False
sentiment                         False
Blob Polarity                     False
Blob Subjectivity                 False
positive Sentiment first half     False
negative Sentiment first half     False
first half sentiment              False
first half Blob Polarity          False
first half Blob Subjectivity      False
positive Sentiment second half    False
negative Sentiment second half    False
second half sentiment             False
second half Blob Polarity         False
second half Blob Subjectivity     False
Capitalization                    False
Topic :                            True
contain_emoji                     False
count_special_chracter            False
count_question_marks              False
subjectivity                      False


In [234]:
x_train=result[['POS_1', 'POS_2', 'POS_3', 'POS_4',
       'Positive Sentiment', 'Negative Sentiment', 'sentiment',
       'Blob Polarity', 'Blob Subjectivity', 'positive Sentiment first half',
       'negative Sentiment first half', 'first half sentiment',
       'first half Blob Polarity', 'first half Blob Subjectivity',
       'positive Sentiment second half', 'negative Sentiment second half',
       'second half sentiment', 'second half Blob Polarity',
       'second half Blob Subjectivity', 'Capitalization', 'Topic :',
       'contain_emoji', 'count_special_chracter', 'count_question_marks',
       'subjectivity', 'count_verbs', 'count_nouns', 'count_pronun',
       'count_adjct', 'count_profane']]

In [203]:
x_test=result_test[['POS_1', 'POS_2', 'POS_3', 'POS_4',
       'Positive Sentiment', 'Negative Sentiment', 'sentiment',
       'Blob Polarity', 'Blob Subjectivity', 'positive Sentiment first half',
       'negative Sentiment first half', 'first half sentiment',
       'first half Blob Polarity', 'first half Blob Subjectivity',
       'positive Sentiment second half', 'negative Sentiment second half',
       'second half sentiment', 'second half Blob Polarity',
       'second half Blob Subjectivity', 'Capitalization', 'Topic :',
       'contain_emoji', 'count_special_chracter', 'count_question_marks',
       'subjectivity', 'count_verbs', 'count_nouns', 'count_pronun',
       'count_adjct', 'count_profane']]

In [204]:
from scipy.sparse import hstack, csr_matrix


In [237]:
x_train.fillna(0,inplace=True)

In [241]:
x_train.isna().any()

POS_1                             False
POS_2                             False
POS_3                             False
POS_4                             False
Positive Sentiment                False
Negative Sentiment                False
sentiment                         False
Blob Polarity                     False
Blob Subjectivity                 False
positive Sentiment first half     False
negative Sentiment first half     False
first half sentiment              False
first half Blob Polarity          False
first half Blob Subjectivity      False
positive Sentiment second half    False
negative Sentiment second half    False
second half sentiment             False
second half Blob Polarity         False
second half Blob Subjectivity     False
Capitalization                    False
Topic :                           False
contain_emoji                     False
count_special_chracter            False
count_question_marks              False
subjectivity                      False


In [243]:
x_test.fillna(0,inplace=True)

In [244]:
x_test.isna().any()

POS_1                             False
POS_2                             False
POS_3                             False
POS_4                             False
Positive Sentiment                False
Negative Sentiment                False
sentiment                         False
Blob Polarity                     False
Blob Subjectivity                 False
positive Sentiment first half     False
negative Sentiment first half     False
first half sentiment              False
first half Blob Polarity          False
first half Blob Subjectivity      False
positive Sentiment second half    False
negative Sentiment second half    False
second half sentiment             False
second half Blob Polarity         False
second half Blob Subjectivity     False
Capitalization                    False
Topic :                           False
contain_emoji                     False
count_special_chracter            False
count_question_marks              False
subjectivity                      False


In [238]:
train_features = hstack([
    train_char_features,
    train_word_features
    ,csr_matrix(np.array(x_train.to_numpy(), dtype=float))], 'csr'
)

In [245]:
test_features = hstack([
    test_char_features,
    test_word_features
    ,csr_matrix(np.array(x_test.to_numpy(), dtype=float))], 'csr'
)

In [246]:
linear_svm_grid(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


          Linear SVM

Accuracy: 0.701
Precision: 0.702
Recall: 0.701
F_score: 0.702

              precision    recall  f1-score   support

           0       0.81      0.80      0.80       531
           1       0.36      0.36      0.36       162

    accuracy                           0.70       693
   macro avg       0.58      0.58      0.58       693
weighted avg       0.70      0.70      0.70       693



In [247]:
logistic_regression_grid(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


 Logistic Regression

Accuracy: 0.680
Precision: 0.690
Recall: 0.680
F_score: 0.685

              precision    recall  f1-score   support

           0       0.80      0.78      0.79       531
           1       0.33      0.36      0.35       162

    accuracy                           0.68       693
   macro avg       0.57      0.57      0.57       693
weighted avg       0.69      0.68      0.68       693



In [248]:
linear_svm(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


          Linear SVM

Accuracy: 0.592
Precision: 0.689
Recall: 0.592
F_score: 0.622

              precision    recall  f1-score   support

           0       0.81      0.61      0.70       531
           1       0.29      0.53      0.38       162

    accuracy                           0.59       693
   macro avg       0.55      0.57      0.54       693
weighted avg       0.69      0.59      0.62       693



In [249]:
logistic_regression(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')


 Logistic Regression

Accuracy: 0.570
Precision: 0.691
Recall: 0.570
F_score: 0.603

              precision    recall  f1-score   support

           0       0.81      0.57      0.67       531
           1       0.29      0.57      0.38       162

    accuracy                           0.57       693
   macro avg       0.55      0.57      0.53       693
weighted avg       0.69      0.57      0.60       693



In [250]:
random_forest(train_features,y_train.astype(int),test_features,y_test.astype(int),class_ratio='balanced')



       Random Forest

Accuracy: 0.690
Precision: 0.677
Recall: 0.690
F_score: 0.683

              precision    recall  f1-score   support

           0       0.79      0.82      0.80       531
           1       0.31      0.28      0.30       162

    accuracy                           0.69       693
   macro avg       0.55      0.55      0.55       693
weighted avg       0.68      0.69      0.68       693



In [251]:
xg_boost(train_features,y_train.astype(int),test_features,y_test.astype(int))



             XGBoost

Accuracy: 0.642
Precision: 0.677
Recall: 0.642
F_score: 0.657

              precision    recall  f1-score   support

           0       0.79      0.72      0.76       531
           1       0.30      0.38      0.33       162

    accuracy                           0.64       693
   macro avg       0.54      0.55      0.54       693
weighted avg       0.68      0.64      0.66       693



In [252]:
Bagging_Classifier_LR(train_features,y_train.astype(int),test_features,y_test.astype(int))



Bagging Calssifier LR

Accuracy: 0.570
Precision: 0.688
Recall: 0.570
F_score: 0.603

              precision    recall  f1-score   support

           0       0.81      0.57      0.67       531
           1       0.29      0.56      0.38       162

    accuracy                           0.57       693
   macro avg       0.55      0.57      0.53       693
weighted avg       0.69      0.57      0.60       693



In [253]:
Bagging_Classifier_SVM(train_features,y_train.astype(int),test_features,y_test.astype(int))



Bagging Calssifier SVM

Accuracy: 0.576
Precision: 0.666
Recall: 0.576
F_score: 0.606

              precision    recall  f1-score   support

           0       0.79      0.61      0.69       531
           1       0.27      0.46      0.34       162

    accuracy                           0.58       693
   macro avg       0.53      0.54      0.51       693
weighted avg       0.67      0.58      0.61       693

